This file is most of the stuff I worked on over the summer, and has a lot of code (most which I've realized is irrelevant) for simulating neutrino oscillation from the surface of the moon to the earth. See "neutrino_travel_to_earth_simulation.ipynb" for the most important stuff

In [4]:
# Import packages
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.integrate import odeint, quad
import math
from scipy.interpolate import interp1d

# Set parameters to make plots look a little nicer
font = {'size' : 15}
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
mpl.rc('font', **font)

In [3]:
#Code from Prof Hyde to find neutrino flavor ratios after oscillating

# Define mass-squared difference (in eV^2) and mixing angle (rad)
Th12 = 34.5*np.pi/180
Th13 = 8.45*np.pi/180
Th23 = 47.7*np.pi/180 # assuming normal mass ordering
Dmsq21 = 7.55 * 10**(-5)
Dmsq31 = 2.50 * 10**(-3) # assuming normal mass ordering


# Return derivative: time (length)-evolution of flavor amplitudes
#density in g / cm^3
def nuDerivVacuum3Flav(amp,x,E_GeV,del13,density): 

    # Set mass values
    mass1 = 0.0
    mass2 = Dmsq21
    mass3 = Dmsq31

    ## Elements of flavor-basis Hamiltonian

    # Mixing matrix (unitary change of basis between mass and flavor)
    # Sines and Cosines of Mixing Angles
    s12 = np.sin(Th12)
    c12 = np.cos(Th12)
    s23 = np.sin(Th23)
    c23 = np.cos(Th23)
    s13 = np.sin(Th13)
    c13 = np.cos(Th13)

    ## Elements of flavor-basis Hamiltonian

    # Mixing matrix
    U12 = np.array([(c12,s12,0),(-s12,c12,0),(0,0,1)])
    U23 = np.array([(1,0,0),(0,c23,s23),(0,-s23,c23)])
    U13 = np.array([(c13,0,s13*np.exp(0.-1.j*del13)),(0,1,0),(-s13*np.exp(0.+1.j*del13),0,c13)],dtype=complex)
    Uint = np.dot(U13,U12)
    U = np.dot(U23,Uint)

    UT = np.transpose(U)
    Udagger = np.conjugate(UT)

    # Mass basis Hamiltonian
    Hmass = np.array([(0,0,0),(0,mass2/(2*E_GeV),0),(0,0,mass3/(2*E_GeV))])

    # Flavor-basis Hamiltonian, without matter
    Hmid = np.dot(Hmass,Udagger)
    Hflav0 = np.dot(U,Hmid)

    # Full flavor-basis Hamiltonian. V_int represents the contribution to
    # potential due to charged-current neutrino-electron interactions.
    # Neutral current factors out of V_int and doesn't affect oscillations.
    vcc = density*0.000193
    Vee = 1.0
    Vint = vcc*np.array([(Vee,0,0),(0,0,0),(0,0,0)],dtype=complex)
    Hflav = 5.1*(Hflav0 + Vint) # factor of 5.1 comes from units... see notes

    # Input array "amp" is four values: the real and imaginary components of
    # each flavor amplitude. Note that "j" instead of "i" is used.
    eAmp = amp[0] + 1.j*amp[1]
    muAmp = amp[2] + 1.j*amp[3]
    tauAmp = amp[4] + 1.j*amp[5]

    # This uses the (flavor-basis) Hamiltonian to define the time-evolution
    deAmpdx = -1.j*(Hflav[0,0]*eAmp + Hflav[0,1]*muAmp + Hflav[0,2]*tauAmp)
    dmuAmpdx = -1.j*(Hflav[1,0]*eAmp + Hflav[1,1]*muAmp + Hflav[1,2]*tauAmp)
    dtauAmpdx = -1.j*(Hflav[2,0]*eAmp + Hflav[2,1]*muAmp + Hflav[2,2]*tauAmp)

    # Return an array of "d(Amplitude)/dx"
    dAmpdx = [np.real(deAmpdx),np.imag(deAmpdx),np.real(dmuAmpdx),np.imag(dmuAmpdx),np.real(dtauAmpdx),np.imag(dtauAmpdx)]
    return dAmpdx

# Return solution for flavor probability as a function of distance.
# Initial state is a 4-component vector of initial flavor amplitudes (real + imag.)
def ThreeFlavorProbs(E_GeV,L_km,del13,density,initial_state):
  x = np.linspace(0.0,L_km,1000)
  solution = odeint(nuDerivVacuum3Flav,initial_state,x,args=(E_GeV,del13,density))
  ProbNuE = solution[:,0]*solution[:,0] + solution[:,1]*solution[:,1]
  ProbNuMu = solution[:,2]*solution[:,2] + solution[:,3]*solution[:,3]
  ProbNuTau = solution[:,4]*solution[:,4] + solution[:,5]*solution[:,5]
  return x,ProbNuE,ProbNuMu,ProbNuTau,solution



In [6]:
#returns the ratio of muon neutrinos to electron neutrinos produced in earths atmosphere
#we assume this is the same for the moon
#for just 2 muon : 1 electron, but i'm leaving this function intact in case we change our midns
def get_initial_flavor_ratios(energy): #GeVs
    return 2 

In [7]:
#convers m to e nue ratio into list of probability amplitudes that we can use in our oscillation code
def turn_flavor_ratios_into_probability_amplitudes(m_to_e_ratio):
    #set initial amplitudes
    m_amp = m_to_e_ratio
    e_amp = 1
    #normalize
    initial_magnitude = math.sqrt(m_amp ** 2 + e_amp ** 2)
    
    m_amp = m_amp / initial_magnitude
    e_amp = e_amp / initial_magnitude
    
    return [e_amp,0,m_amp,0,0,0]
    

In [8]:
#returns probability amplitudes of initial neutrino flux for a given energy
def get_prob_amplitudes(energy):
    ratios = get_initial_flavor_ratios(energy)
    return turn_flavor_ratios_into_probability_amplitudes(ratios)

In [9]:
#the decoherence_travel_earth function is far faster so use that instead

#simulates neutrino oscillation as neutrino travels from moon to earth
#returns a list of similar to the output of ThreeFlavorProbs (x,ProbNuE,ProbNuMu,ProbNuTau,solution)
def neutrino_travel_to_earth(energy, three_flavor_prob_output):
    previous_travel_data = []
    for i in range(len(three_flavor_prob_output)):
        previous_travel_data.append(three_flavor_prob_output[i])    
    
    flavor_amplitudes = previous_travel_data[4][-1]
    #print(previous_travel_data[1][0])
    distance_to_earth = 384000
    after_travel_to_earth = ThreeFlavorProbs(energy,distance_to_earth,np.pi/3,0,flavor_amplitudes)
        
    for i in range(len(after_travel_to_earth[0])):
        after_travel_to_earth[0][i] += previous_travel_data[0][-1]
    
    total_journey = []
    for i in range(len(previous_travel_data)):
        concatenated_data = np.concatenate([previous_travel_data[i], after_travel_to_earth[i]])
        total_journey.append(concatenated_data)
    return total_journey

In [10]:
#greates a graph of flavor probabilities as a function of distance
def visualize_oscillation(three_flavor_prob_output):
    plt.plot(three_flavor_prob_output[0], three_flavor_prob_output[1], label=r'$P_e$')   
    plt.plot(three_flavor_prob_output[0], three_flavor_prob_output[2], label=r'$P_{\mu}$')
    plt.plot(three_flavor_prob_output[0], three_flavor_prob_output[3], label=r'$P_{\tau}$')
    plt.legend(loc='best')
    plt.xlabel('L (km)')
    plt.ylabel('Probability')
    return


In [11]:
#prints out the final flavor probabilities
def show_final_probabilities(three_flavor_prob_output):
    print("electron probability: " + str(three_flavor_prob_output[1][-1]))
    print("muon probability: " + str(three_flavor_prob_output[2][-1]))
    print("tau probability: " + str(three_flavor_prob_output[3][-1]))

In [12]:
# gives oscillation derivative, but specialized for moon.
# x, y, z is location relative to center of moon
# density function takes this location and returns a density
def moon_deriv_three_flavor(amp,x,E_GeV,del13,y,z, density_function):

    # Set mass values
    mass1 = 0.0
    mass2 = Dmsq21
    mass3 = Dmsq31

    ## Elements of flavor-basis Hamiltonian

    # Mixing matrix (unitary change of basis between mass and flavor)
    # Sines and Cosines of Mixing Angles
    s12 = np.sin(Th12)
    c12 = np.cos(Th12)
    s23 = np.sin(Th23)
    c23 = np.cos(Th23)
    s13 = np.sin(Th13)
    c13 = np.cos(Th13)

    ## Elements of flavor-basis Hamiltonian

    # Mixing matrix
    U12 = np.array([(c12,s12,0),(-s12,c12,0),(0,0,1)])
    U23 = np.array([(1,0,0),(0,c23,s23),(0,-s23,c23)])
    U13 = np.array([(c13,0,s13*np.exp(0.-1.j*del13)),(0,1,0),(-s13*np.exp(0.+1.j*del13),0,c13)],dtype=complex)
    Uint = np.dot(U13,U12)
    U = np.dot(U23,Uint)

    UT = np.transpose(U)
    Udagger = np.conjugate(UT)

    # Mass basis Hamiltonian
    Hmass = np.array([(0,0,0),(0,mass2/(2*E_GeV),0),(0,0,mass3/(2*E_GeV))])

    # Flavor-basis Hamiltonian, without matter
    Hmid = np.dot(Hmass,Udagger)
    Hflav0 = np.dot(U,Hmid)

    # Full flavor-basis Hamiltonian. V_int represents the contribution to
    # potential due to charged-current neutrino-electron interactions.
    # Neutral current factors out of V_int and doesn't affect oscillations.
    density = density_function(x, y, z)
    vcc = density*0.000193
    Vee = 1.0
    Vint = vcc*np.array([(Vee,0,0),(0,0,0),(0,0,0)],dtype=complex)
    Hflav = 5.1*(Hflav0 + Vint) # factor of 5.1 comes from units... see notes

    # Input array "amp" is four values: the real and imaginary components of
    # each flavor amplitude. Note that "j" instead of "i" is used.
    eAmp = amp[0] + 1.j*amp[1]
    muAmp = amp[2] + 1.j*amp[3]
    tauAmp = amp[4] + 1.j*amp[5]

    # This uses the (flavor-basis) Hamiltonian to define the time-evolution
    deAmpdx = -1.j*(Hflav[0,0]*eAmp + Hflav[0,1]*muAmp + Hflav[0,2]*tauAmp)
    dmuAmpdx = -1.j*(Hflav[1,0]*eAmp + Hflav[1,1]*muAmp + Hflav[1,2]*tauAmp)
    dtauAmpdx = -1.j*(Hflav[2,0]*eAmp + Hflav[2,1]*muAmp + Hflav[2,2]*tauAmp)

    # Return an array of "d(Amplitude)/dx"
    dAmpdx = [np.real(deAmpdx),np.imag(deAmpdx),np.real(dmuAmpdx),np.imag(dmuAmpdx),np.real(dtauAmpdx),np.imag(dtauAmpdx)]
    return dAmpdx


In [13]:
#simulates neutrino oscillation, but specialized for moon with variable density
#y, z are fixed values, while neutrino travels parallel to x-axis
# density function takes x, y, z (location relative to center of moon) and returns moon's density in g/cm^3
def three_flavor_prob_any_moon_density(energy, y, z, density_function): #energy in GeVs, y and z in km
    r_moon = 1737 #km
    x_final = np.sqrt(r_moon**2 - y**2 - z**2)
    x0 = -x_final
    
    x = np.linspace(x0,x_final,1000)
    del13 = np.pi/3
    solution = odeint(moon_deriv_three_flavor,get_prob_amplitudes(energy),x,args=(energy,del13,y, z, density_function))
    ProbNuE = solution[:,0]*solution[:,0] + solution[:,1]*solution[:,1]
    ProbNuMu = solution[:,2]*solution[:,2] + solution[:,3]*solution[:,3]
    ProbNuTau = solution[:,4]*solution[:,4] + solution[:,5]*solution[:,5]
    return x,ProbNuE,ProbNuMu,ProbNuTau,solution 

In [14]:
#this returns a density function that gives density as a function of x, y, z for a given core-mantle density ratio
def make_density_function_from_ratio(core_mantle_density_ratio, core_radius): #core radius in km
    r_moon = 1737  - 10#km
    r_moon_cm = r_moon * 10 ** 5
    crust_to_core_distance = 1700 * 10 ** 5 # cm
    r_core_cm = core_radius * 10 ** 5 # cm
    
    
    m_moon = 7.342 * 10 ** 25 #grams
    m_crust = (4/3) * math.pi * (r_moon_cm ** 3 - crust_to_core_distance ** 3)
    m_moon_without_crust = m_moon - m_crust
    V_core = (4/3) * math.pi * (r_core_cm ** 3)
    V_mantle = (4/3) * math.pi * (crust_to_core_distance ** 3 - r_core_cm)
    
    mantle_density = m_moon_without_crust / (V_core * core_mantle_density_ratio + V_mantle)
    core_density = mantle_density * core_mantle_density_ratio
        
    
    def density_function(x, y, z, d_core=core_density, d_mantle=mantle_density):
        r = np.sqrt(x**2 + y**2 + z**2)
        if r < core_radius: #core
            #print("core denstiy" + str(d_core))
            return d_core
        if r < 1700: #mantle
            #print("mantle denstiy" + str(d_mantle))
            return d_mantle
        else:
            return 2.55 # crust density is known pretty accurately
        
    return density_function

In [15]:
#models neutrino travel to earth using approximation form gaisser textbook
def decoherence_travel_earth(e_prob,m_prob,t_prob):
    p = ([.55, .25, .2],
         [0.25, 0.37, 0.38],
         [0.2, 0.38, 0.42])
    initial_ratios = ([e_prob,m_prob,t_prob])
    product = np.matmul(p, initial_ratios)
    return product

In [16]:
#this function simulates many neutrinos of a given energy hitting the moon at different locations and oscillating through it to earth
#then it returns the three flavor probabilities
def monte_carlo_uniform_core(energy, core_mantle_density_ratio, core_radius): #energy in GeVs
    r_moon = 1737 #km
    
    density_function = make_density_function_from_ratio(core_mantle_density_ratio, core_radius)
    
    e_prob = 0
    m_prob = 0
    t_prob = 0
    scale_factor = 0

    radii = np.linspace(0,r_moon,50)
    for radius in radii:
        after_moon = three_flavor_prob_any_moon_density(energy, radius, 0, density_function)
        e_prob += after_moon[1][-1] * radius 
        m_prob += after_moon[2][-1] * radius
        t_prob += after_moon[3][-1] * radius
        scale_factor+=radius
    
    #normalize probabilities
    e_prob /= scale_factor
    m_prob /= scale_factor 
    t_prob /= scale_factor
    
    #now need to propogate to earth
    at_earth_ratios = decoherence_travel_earth(e_prob, m_prob, t_prob)
    return at_earth_ratios

In [17]:
# this visualises the effect of core-mantle density ratio on neutrino flavor probabilities
def visualize_density_ratio_vs_probabilities(energy): #energy in GeVs
    ratios = [1, 2, 3, 5, 7, 10, 20, 25, 50, 75, 100] #range of density ratios to test
    elec_probabilities = []
    muon_probabilities = []
    tau_probabilities = []
    for ratio in ratios:
        new_probabilities = monte_carlo_uniform_core(energy, ratio, 330)
        elec_probabilities.append(new_probabilities[0])
        muon_probabilities.append(new_probabilities[1])
        tau_probabilities.append(new_probabilities[2])
        
    plt.plot(ratios,elec_probabilities,label=r'$P_e$', linestyle=':')
    plt.plot(ratios,muon_probabilities,label=r'$P_{\mu}$', linestyle='-')
    plt.plot(ratios,tau_probabilities,label=r'$P_{\tau}$', linestyle='--')
    plt.legend(loc='best')
    plt.xlabel('Moon-Mantle Density Ratio')
    plt.ylabel('Flavor Probability')
    plt.annotate(r'$E_{\nu} = 1$ GeV', xy=(0.5, 1.03), xycoords='axes fraction', ha='center')#, fontsize=12)
    plt.savefig("core_density_vs_nu_flavor",dpi=300,bbox_inches='tight')
    return

In [18]:
def make_density_ratio_vs_energy_heatmap_data():
    energies = np.linspace(1, 10, 3)#[1, 2, 3, 5, 7, 10, 20, 50, 75, 100]
    ratios = np.linspace(0, 5, 3)
    elec_probabilities = []
    muon_probabilities = []
    tau_probabilities = []
    for energy in energies:
        for ratio in ratios:
            new_probabilities = monte_carlo_uniform_core(energy, ratio, 330)
            elec_probabilities.append(new_probabilities[0])
            muon_probabilities.append(new_probabilities[1])
            tau_probabilities.append(new_probabilities[2])
    return elec_probabilities, muon_probabilities, tau_probabilities

In [19]:
#creates a heatmap to display the muon probabilities for a given energy and core-mantle density ratio
#takes data from make_density_ratio_vs_energy_heatmap_data
def visualize_density_ratio_and_energy_heatmap(muon_probabilities):
    """energies = np.linspace(1, 10, 100)#[1, 2, 3, 5, 7, 10, 20, 50, 75, 100]
    ratios = np.linspace(1, 100, 100)#[1, 2, 3, 5, 7, 10, 20, 50, 75, 100]
    """
    
    energies = np.linspace(1, 10, 3)#[1, 2, 3, 5, 7, 10, 20, 50, 75, 100]
    ratios = np.linspace(0, 5, 3)
    
    # Create a meshgrid
    X, Y = np.meshgrid(energies, ratios)     
    # Plot the heatmap
    plt.figure(figsize=(8, 6))
    plt.pcolormesh(X, Y, muon_probabilities, shading='auto', cmap='viridis')
    plt.colorbar(label='Muon Flavor Probability')
    plt.xlabel('Core-Mantle Density Ratio')
    plt.ylabel('Energies (GeV)')
    plt.title('Muon Flavor Probability') 
    plt.savefig("Density_Ratio_heatmap_layered_core",dpi=300,bbox_inches='tight')

In [20]:
#e_probabilities, m_probabilities, t_probabilities = make_density_ratio_vs_energy_heatmap_data()

In [21]:
#visualize_density_ratio_and_energy_heatmap(np.array(m_probabilities).reshape(3, 3))

In [22]:
#makes density function from ratio like other function
#but assumes core has two layers!
def make_density_function_from_ratio_layered_core(core_mantle_density_ratio, inner_outer_core_density_ratio):
    r_moon = 1737  - 10#km
    r_moon_cm = r_moon * 10 ** 5
    crust_to_core_distance = 1700 * 10 ** 5 # cm
    r_core = 330 * 10 ** 5 # cm
    
    
    m_moon = 7.342 * 10 ** 25 #grams
    m_crust = (4/3) * math.pi * (r_moon_cm ** 3 - crust_to_core_distance ** 3)
    m_moon_without_crust = m_moon - m_crust
    V_core = (4/3) * math.pi * (r_core ** 3)
    V_mantle = (4/3) * math.pi * (crust_to_core_distance ** 3 - r_core)
    
    mantle_density = m_moon_without_crust / (V_core * core_mantle_density_ratio + V_mantle)
    overall_core_density = mantle_density * core_mantle_density_ratio
    
    
    m_core = V_core * overall_core_density
    r_inner_core = 240 * 10 ** 5 #cm
    r_outer_core = r_core + r_inner_core
    
    V_inner_core = (4/3) * math.pi * (r_core ** 3)
    V_outer_core = V_core - V_inner_core
    
    outer_core_density = m_core / (V_core * inner_outer_core_density_ratio + V_inner_core)
    inner_core_density = outer_core_density * inner_outer_core_density_ratio
        
    
    def density_function(x, y, z, d_inner = inner_core_density, d_outer = outer_core_density, d_mantle=mantle_density):
        r = np.sqrt(x**2 + y**2 + z**2)
        if r < 240:
            return d_inner
        if r < 330: #core
            return d_outer
        if r < 1700: #mantle
            return d_mantle
        else:
            return 2.55 # crust density is known pretty accurately
        
    return density_function

In [23]:
#same as monte carlo uniform core function
#but assumes core has layers
#turns out these layers don't seem to matter!
def monte_carlo_layered_core(energy, core_mantle_density_ratio, inner_outer_core_density_ratio):
    r_moon = 1737 #km
    
    density_function = make_density_function_from_ratio_layered_core(core_mantle_density_ratio, inner_outer_core_density_ratio)
    
    e_prob = 0
    m_prob = 0
    t_prob = 0
    scale_factor = 0

    radii = np.linspace(0,r_moon,50)
    for radius in radii:
        after_moon = three_flavor_prob_any_moon_density(energy, radius, 0, density_function)
        after_traveling_to_earth = neutrino_travel_to_earth(energy, after_moon)
        e_prob += after_traveling_to_earth[1][-1] * radius 
        m_prob += after_traveling_to_earth[2][-1] * radius
        t_prob += after_traveling_to_earth[3][-1] * radius
        scale_factor+=radius
    
    #normalize probabilities
    e_prob /= scale_factor
    m_prob /= scale_factor 
    t_prob /= scale_factor
    return e_prob,m_prob,t_prob  

In [24]:
#finds flavor probabilities for a range of energies and ratios
#I used this function to make an interesting heatmap
def make_density_ratio_vs_energy_heatmap_data_layered_core():
    energies = np.linspace(1, 10, 100)#[1, 2, 3, 5, 7, 10, 20, 50, 75, 100]
    ratios = np.linspace(1, 100, 100)#[1, 2, 3, 5, 7, 10, 20, 50, 75, 100]
    elec_probabilities = []
    muon_probabilities = []
    tau_probabilities = []
    for energy in energies:
        for ratio in ratios:
            new_probabilities = monte_carlo_layered_core(energy, ratio, 2)
            elec_probabilities.append(new_probabilities[0])
            muon_probabilities.append(new_probabilities[1])
            tau_probabilities.append(new_probabilities[2])
    return elec_probabilities, muon_probabilities, tau_probabilities

In [25]:
#returns final electron flux, muon flux, and electron to muon flavor ration after neutrinos have gone through moon and to eart
#for a given lunar model
def make_final_flavor_flux(mantle_core_density_ratio, core_radius):
    log_energies = np.linspace(0, 1.9, 30) #GeVs
    muon_fluxes = []
    electron_fluxes = []
    e_to_m_ratios = []
    for log_energy in log_energies:
        total_flux = get_diff_interactions(log_energy)
        monte_carlo_results = monte_carlo_uniform_core(10 ** log_energy, mantle_core_density_ratio, core_radius) 
        electron_flux = total_flux * monte_carlo_results[0] #* energy
        electron_fluxes.append(electron_flux)
        muon_flux = total_flux * monte_carlo_results[1] #* energy
        muon_fluxes.append(muon_flux)
        e_to_m_ratios.append(electron_flux / muon_flux)
    
    electron_flux_interp = interp1d(log_energies, electron_fluxes, kind='linear')
    muon_flux_interp = interp1d(log_energies, muon_fluxes, kind='linear')
    e_to_m_interp = interp1d(log_energies, e_to_m_ratios, kind='linear')
    
    return electron_flux_interp, muon_flux_interp, e_to_m_interp

In [2]:
# for a given core radius and core-mantle density ratio, returns the density of the core
def get_core_density(core_radius, core_mantle_density_ratio):
    r_moon = 1737  - 10#km
    r_moon_cm = r_moon * 10 ** 5
    crust_to_core_distance = 1700 * 10 ** 5 # cm
    r_core_cm = core_radius * 10 ** 5 # cm
    
    
    m_moon = 7.342 * 10 ** 25 #grams
    m_crust = (4/3) * math.pi * (r_moon_cm ** 3 - crust_to_core_distance ** 3)
    m_moon_without_crust = m_moon - m_crust
    V_core = (4/3) * math.pi * (r_core_cm ** 3)
    V_mantle = (4/3) * math.pi * (crust_to_core_distance ** 3 - r_core_cm)
    
    mantle_density = m_moon_without_crust / (V_core * core_mantle_density_ratio + V_mantle)
    core_density = mantle_density * core_mantle_density_ratio
    return core_density

In [10]:
get_core_density(330, 1.77)

6.149395559642964